# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [10]:
context = [
    {"role": "system", "content": """
You are a bot that generates SQLite SQL.

RULES:
- Return ONLY the SQL query (no explanations, no markdown, no backticks).
- Use only the tables/columns provided below.
- If the request cannot be answered with this schema, return exactly: NEED_MORE_SCHEMA
    """},

    {"role": "system", "content": """
TABLE employees(
  ID_usr INTEGER,
  name TEXT
);

TABLE salary(
  ID_usr INTEGER,
  year INTEGER,
  salary REAL
);

TABLE studies(
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years INTEGER,
  Speciality TEXT
);
    """}
]

In [11]:
context.append({
    "role": "system",
    "content": """
EXAMPLES (convert question -> SQL):

Q: Show each employee name with their salary for year 2023.
A: SELECT e.name, s.salary
   FROM employees e
   JOIN salary s ON e.ID_usr = s.ID_usr
   WHERE s.year = 2023;

Q: List the top 5 highest salaries with employee names (any year).
A: SELECT e.name, s.year, s.salary
   FROM employees e
   JOIN salary s ON e.ID_usr = s.ID_usr
   ORDER BY s.salary DESC
   LIMIT 5;

Q: Show employees who studied at 'Harvard'.
A: SELECT DISTINCT e.name
   FROM employees e
   JOIN studies st ON e.ID_usr = st.ID_usr
   WHERE st.Institution = 'Harvard';
"""
})

In [12]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [14]:
#new
context_user = context.copy()
print(return_CCRMSQL("Show each employee name with their salary for year 2023.", context_user))

SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year = 2023;


In [15]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Show each employee name with their salary for year 2023.", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year = '2023';
```

This SQL query retrieves the name of each employee along with their salary for the year 2023 by joining the "employees" table with the "salary" table on the employee ID and filtering the results to only include salaries for the year 2023.


In [16]:
#new
print(return_CCRMSQL("Show the average salary per year, sorted by year.", context_user))

SELECT year, AVG(salary) AS average_salary
FROM salary
GROUP BY year
ORDER BY year;


In [17]:
#old
print(return_CCRMSQL("Show the average salary per year, sorted by year.", old_context_user))

This is your SQL:
```sql
SELECT year, AVG(salary) AS average_salary
FROM salary
GROUP BY year
ORDER BY year;
```

This SQL query selects the year and calculates the average salary for each year from the "salary" table. It groups the results by year, calculates the average salary for each year, and then sorts the results in ascending order by year.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

# NL2SQL Prompt Variations — Findings Report
Schema used across tests: employees, salary, studies (SQLite).

## Setup
Goal: compare prompt styles for converting natural language questions into valid SQLite SQL.
I tested multiple prompts using the same tables to keep the comparison fair.

## Version 1 (Old / baseline prompt)
Behavior: often outputs SQL plus extra explanation and sometimes code fences.
Pro: easy to read.
Con: noisy if you need SQL-only output for automation.

## Version 2 (New prompt: schema + SQL-only rule)
Behavior: usually returns only the SQL query.
Pro: clean output that’s easy to reuse.
Con: if the schema is missing from context, it asks for tables instead of generating SQL.

## Version 3 (New prompt + few-shot examples + refusal rule)
Behavior: most consistent SQL-only output.
Pro: few-shot examples improve formatting consistency.
Pro: refusal rule reduces hallucination when schema is insufficient (NEED_MORE_SCHEMA).

## Variations that didn’t work well
- When table definitions weren’t included (or I reused a stale context copy), the model asked for tables instead of producing SQL.
- The old prompt sometimes mixed SQL with explanation/formatting, which is inconvenient for pipelines expecting raw SQL.

## What I learned
- The schema must be in the prompt context for reliable NL2SQL.
- “Return ONLY SQL” is a strong constraint that improves usability.
- Few-shot examples help keep outputs consistent.
- Recreating `context_user = context.copy()` per test cell avoids stale-context errors.

## Conclusion
Best overall: Version 3 (schema + SQL-only + few-shot + refusal rule).
It produced the cleanest outputs and reduced failure cases / hallucinations.